## Preprocessing the Dataset

In [34]:
import os
import pandas as pd

# read the file in the folder
path = os.listdir('./redbus_details/')
file_path = [ os.path.join("./redbus_details/", file) for file in path]

df = [ pd.read_csv(file) for file in file_path]

# concatenate the all dataframe 
df_comb = pd.concat(df, ignore_index=True)

# Remove the INR in the price column
df_comb['Price'] = df_comb['Price'].str.replace('INR', '')

# Extract the numeric value in the seat_availability column
df_comb['Seat_Availability'] = df_comb['Seat_Availability'].str.extract(r"(\d+)")


# Add the id column to the dataset at first
df_comb.insert(loc=0, column='id', value=range(1, len(df_comb)+1))

df_comb.to_csv("redbus_routes.csv", index=False)


## Store the data into database

In [1]:
from sqlite3 import connect

conn = connect('redbus_db.db')
curr = conn.cursor()

In [9]:
import pandas as pd
df = pd.read_csv('./redbus_routes.csv')




In [67]:
# create a table
table = """
    CREATE TABLE redbus_details (
        id INT PRIMARY KEY,
        bus_name VARCHAR(255),
        route_name VARCHAR(255),
        bus_type VARCHAR(255),
        departing_time VARCHAR(150),
        duration VARCHAR(100),
        reaching_time VARCHAR(150),
        star_rating  DECIMAL(3,2),
        price DECIMAL(10,2),
        seat_availability INT,
        route_link VARCHAR(255)  
    );
"""

In [80]:
curr.execute(table)

In [100]:
# insert the data into the table
sql_query = """
    INSERT INTO redbus_details (
        id, bus_name, route_name, bus_type, departing_time,
        duration, reaching_time, star_rating, price, seat_availability, route_link
    ) VALUES 
    (:id, :bus_name, :route_name, :bus_type, :departing_time,
    :duration, :reaching_time, :star_rating, :price, :seat_availability, :route_link);
"""


# for index,row in df.iterrows():
#     curr.execute(
#        sql_query,
#         (
#             row['id'],
#             row['Bus_Name'],
#             row['Route_Name'],
#             row['Bus_Type'],
#             row['Departing_Time'],
#             row['Duration'],
#             row['Reaching_Time'],
#             row['Star_Rating'],
#             row['Price'],
#             row['Seat_Availability'],
#             row['Route_Link']
#         )
#     )
#     conn.commit()
    
for index, row in df.iterrows():
    params = {
        'id': row['id'],
        'bus_name': row['Bus_Name'],
        'route_name': row['Route_Name'],
        'bus_type': row['Bus_Type'],
        'departing_time': row['Departing_Time'],
        'duration': row['Duration'],
        'reaching_time': row['Reaching_Time'],
        'star_rating': row['Star_Rating'],
        'price': row['Price'],
        'seat_availability': row['Seat_Availability'],
        'route_link': row['Route_Link']
    }
    curr.execute(sql_query, params)
    conn.commit()
    

In [97]:
# curr.execute("delete from  redbus_details;")

In [3]:
import pandas as pd

In [25]:
data = pd.read_sql_query("select * from redbus_details order by price asc ;", conn)
data['bus_type'].unique()



array(['Non AC Seater (2+3)', 'Volvo A/C Seater (2+2)',
       'Volvo AC Seater 2+2', 'AC Seater (2+2)',
       'Express Non AC Seater 2+3', 'Ordinary Non AC Seater 2+3',
       'Semi Deluxe Non AC Seater 2+2', 'Rajdhani Non AC Seater 2+3',
       'Express(Non AC Seater)', 'AC Seater (2+3)', 'A/C Sleeper (2+1)',
       'A/C Seater (2+3)', 'A/C Seater (2+1)',
       'A/C Seater / Sleeper (2+1)', 'A/C Seater (2+2)',
       'SAPTAGIRI EXPRESS', 'NON A/C Seater Push Back (2+2)',
       'NON A/C Seater (2+2)', 'Bharat Benz A/C Seater (2+2)',
       'A/C Executive (2+3)', 'Super Fast Non AC Seater (2+3)',
       'NON AC Seater / Sleeper 2+1', 'Janrath AC Seater 2+3',
       'A/C Seater / Sleeper (2+2)', 'Non A/C Seater / Sleeper (2+1)',
       'Super Express Non AC Seater Air Bus (2+2)',
       'Express (Non AC Seater 2+2)',
       'Super Luxury Volvo AC Seater Pushback 2+2',
       'NON A/C Sleeper (2+1)', 'Janrath AC Seater 2+2',
       'Pink Express AC Seater 2+2', 'Swift Deluxe Non AC Ai

In [ ]:
# Check for keywords and create new columns
df['is_sleeper'] = df['bus_type'].str.contains('Sleeper', case=False)
df['is_seater'] = df['bus_type'].str.contains('Seater', case=False)
df['is_ac'] = df['bus_type'].str.contains('AC|A/C', case=False)
df['is_non_ac'] = df['bus_type'].str.contains('Non AC|NON A/C', case=False)

In [27]:
bustype =data['bus_type'].unique()
bustype

In [29]:
aa = data[data['bus_type'].str.contains('Seater', case=False)]
aa

,id,bus_name,route_name,bus_type,departing_time,duration,reaching_time,star_rating,price,seat_availability,route_link
0,5031,SBSTC-DURGAPUR - ROHINI - 12:40 (KULTEKRY) DGP...,Durgapur (West Bengal) to Bankura,Non AC Seater (2+3),13:15,10h 45m,00:00,3.0,48.0,52,https://www.redbus.in/bus-tickets/durgapur-to-...
1,5033,SBSTC-BERHAMPUR - MIDNAPUR - VIA - SURI / BNK ...,Durgapur (West Bengal) to Bankura,Non AC Seater (2+3),14:00,10h 00m,00:00,3.0,48.0,52,https://www.redbus.in/bus-tickets/durgapur-to-...
2,5034,SBSTC-BERHAMPUR - PURULIA - VIA - BANKURA - 08...,Durgapur (West Bengal) to Bankura,Non AC Seater (2+3),14:15,09h 45m,00:00,3.0,48.0,30,https://www.redbus.in/bus-tickets/durgapur-to-...
3,5032,SBSTC-DURGAPUR - JHARGRAM - 13:30 (DGP DEPOT) ...,Durgapur (West Bengal) to Bankura,Non AC Seater (2+3),13:30,01h 00m,14:30,3.0,53.0,52,https://www.redbus.in/bus-tickets/durgapur-to-...
4,5035,SBSTC-DURGAPUR - JHARGRAM - 14:30 (JHR DEPOT) ...,Durgapur (West Bengal) to Bankura,Non AC Seater (2+3),14:30,01h 00m,15:30,3.0,53.0,51,https://www.redbus.in/bus-tickets/durgapur-to-...
...,...,...,...,...,...,...,...,...,...,...,...
9432,9289,IntrCity SmartBus,Lucknow to Mathura,Bharat Benz A/C Seater /Sleeper (2+1),22:45,05h 45m,04:30,4.7,5700.0,46,https://www.redbus.in/bus-tickets/lucknow-to-m...
9433,745,Raja Buses,Vinukonda to Hyderabad,NON AC Seater / Sleeper 2+1,22:50,08h 00m,06:50,0.0,5999.0,2,https://www.redbus.in/bus-tickets/vinukonda-to...
9434,6507,IntrCity SmartBus,Lucknow to Delhi,Bharat Benz A/C Seater /Sleeper (2+1),22:45,08h 45m,07:30,4.5,6000.0,46,https://www.redbus.in/bus-tickets/lucknow-to-d...
9435,8122,IntrCity SmartBus,Lucknow to Delhi,Bharat Benz A/C Seater /Sleeper (2+1),22:45,08h 45m,07:30,4.5,6000.0,46,https://www.redbus.in/bus-tickets/lucknow-to-d...
